In [1]:
! pip install econml
import numpy as np
import pandas as pd
from econml.dml import CausalForestDML

In [2]:
# 创建示例数据集
np.random.seed(42)
n_days = 100
data = pd.DataFrame({
    'date': pd.date_range(start='2024-01-01', periods=100),
    'marketing_campaign': ['A', 'B', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A'] + ['C'] * 90,
    'weather': ['cloudy', 'rainy', 'rainy', 'rainy', 'rainy'] + ['sunny'] * 95,
    'visitors': [1795, 2051, 2643, 4627, 2363] + [3799, 3191, 1608, 4272, 2147] * 19,
    'conversion_rate': [0.035246, 0.040874, 0.045751, 0.029779, 0.026607] + [0.030759, 0.027625, 0.028859, 0.029677, 0.045460] * 19
})

# 增加一个人为引入的因果关系 - 假设某个营销活动在第50天开始
data.loc[data['date'] >= '2024-02-20', 'marketing_campaign'] = 'A'

# 定义因果模型并拟合数据
causal_model = CausalForestDML(
    model_y={'model': 'linear', 'fit_cate_intercept': True, 'max_iter': 10000}, 
    model_t={'model': 'linear', 'fit_cate_intercept': True, 'max_iter': 10000}, 
    discrete_treatment=True
)
X = data[['marketing_campaign', 'weather']]
T = pd.get_dummies(data['marketing_campaign'], drop_first=True)
# Align dimensions by selecting the same number of rows from Y, T, and X
y = data['visitors'].iloc[:len(T)]
X = X.iloc[:len(T)]
print("Shape of T (Treatment):", T.shape)
print("Shape of y (Outcome):", y.shape)
print("Shape of X (Features):", X.shape)

causal_model.fit(y, T, X=X)  # Pass X as a keyword argument

# 估计因果效应
effect = causal_model.effect(X)
print("Estimated Causal Effect:", effect)

Shape of T (Treatment): (100, 2)
Shape of y (Outcome): (100,)
Shape of X (Features): (100, 2)


ValueError: Found input variables with inconsistent numbers of samples: [100, 200]